<center> <img src="https://impassivewealth.com/wp-content/uploads/2020/10/ActivePassive-768x241.png" align="center" width="960" height="480"/> </center>

<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> I.F. Blas César Sánchez Barba </font> <br> <br>
   
<center> <font color= #555555> <font size = 4> Jun 2021 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/if723286/Portafolio-Activo-Pasivo'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Lab 1 </font> <br> <br> <font color= #555555> <font size = 5> Active Investment vs Pasive Investment. Which one is better?  </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  Include here a description of the whole work, use no more than 400 words, you can use **Bold** and *Italic* highlights, also you can inclue [links](www.iteso.mx). Usually is a good strategy to generate this text at the very end of the process, since you have to include all aspects of the work, from a brief introduction, to problem definitions and methods used, to finally the results found. </font>

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

<hr style="border:0.02in solid gray"> </hr>

This is an introduction, normally, it is done at the very end, when all the other content in the notebook is completed but just before the abstract (which is short version of the introduction) ... 

<br>

# <font color= #6B6B6B> 2. Install/Load Packages and Depedencies </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

Finally, from the notebook it will be possible to acces other files within the mounted folder

### <font color= #6B6B6B> 2.1 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas>=1.1.1 
- numpy>=1.19.1
- jupyter>=1.0.0
- _Add any other here_

In [1222]:
# Install all the pip packages in the requirements.txt
%%capture
import sys
!{sys.executable} -m pip install -r requirements.txt

UsageError: Line magic function `%%capture` not found.


### <font color= #6B6B6B> 2.2 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

- files/precios.csv : Precios históricos
- files/datos.txt : Otros datos históricos

<br>

# <font color= #6B6B6B> 3. Data Description </font>

<hr style="border:0.02in solid gray"> </hr>

Here we explain the data

### <font color= #6B6B6B> 3.1 Selection of stocks for portfolio </font> <font color= #555555>

We want to create our own portfolio. In order to do that is necesarry ti know which stocks are the most important ones in the NAFTRAC.

Since NAFRAC is a ETF that replies the S&P IPC 35, and the S&P IPC includes only the most strong enterprices in the Mexican market based in its level of capitalization, we need to obtain those stock that always appear in the NAFTRAC

First of all, lets get all the stocks that appered at lest once during the last two years

In [1223]:
import pandas as pd
import os
import pandas as pd

# directory containing the csv files
directory = 'files'



columns = ['Ticker']
df_global = pd.DataFrame(columns=columns)

# loop through all the files in the directory
for filename in os.listdir(directory):
    # check if the file is a csv file
    if filename.endswith(".csv"):
        # read the csv file and store it in a data frame
        df = pd.read_csv(os.path.join(directory, filename), skiprows=2, nrows=37 , usecols=["Ticker", 'Peso (%)', 'Precio'])
        # append the data frame to the list
        df_global = pd.concat([df_global, df])      
df_global

,Ticker,Peso (%),Precio
0,AMXL,12.29,13.67
1,WALMEX*,11.14,58.35
2,GMEXICOB,10.57,87.84
3,FEMSAUBD,9.79,139.70
4,GFNORTEO,9.39,101.57
...,...,...,...
32,Q*,0.60,94.95
33,BOLSAA,0.56,39.53
34,VOLARA,0.56,21.35
35,MEGACPO,0.52,57.87


Now we want to obtain how many times each stock appear in the NAFTRAC

In [1224]:
df_global['Ticker'] = df_global['Ticker'].map(lambda x: x.replace('*',''))
df_global['Ticker'] = df_global['Ticker'].map(lambda x: x.replace('.',''))
df_global['Ticker'] = df_global['Ticker'].apply(lambda x: "{}{}".format(x, '.MX'))

In [1225]:
occurrences = df_global['Ticker'].value_counts()
occurrences

AMXL.MX          25
GFINBURO.MX      25
MXN.MX           25
ALSEA.MX         25
LIVEPOLC1.MX     25
BBAJIOO.MX       25
Q.MX             25
GCC.MX           25
VESTA.MX         25
BOLSAA.MX        25
MEGACPO.MX       25
GCARSOA1.MX      25
WALMEX.MX        25
PINFRA.MX        25
ALFAA.MX         25
PE&OLES.MX       25
CUERVO.MX        25
AC.MX            25
TLEVISACPO.MX    25
GMEXICOB.MX      25
FEMSAUBD.MX      25
GFNORTEO.MX      25
OMAB.MX          25
GAPB.MX          25
ELEKTRA.MX       25
CEMEXCPO.MX      25
ASURB.MX         25
GRUMAB.MX        25
BIMBOA.MX        25
KOFUBL.MX        25
ORBIA.MX         25
KIMBERA.MX       25
RA.MX            23
LABB.MX          20
 .MX             17
VOLARA.MX        17
SITESB1.MX       14
SITES1A-1.MX     11
GENTERA.MX        7
IENOVA.MX         4
LASITEB1.MX       4
NMKA.MX           2
USD.MX            1
LASITE.MX         1
Name: Ticker, dtype: int64

Then lets just select those stocks that appear all and every single time in the NAFTRAC

In [1226]:
occurrences.head(33)

AMXL.MX          25
GFINBURO.MX      25
MXN.MX           25
ALSEA.MX         25
LIVEPOLC1.MX     25
BBAJIOO.MX       25
Q.MX             25
GCC.MX           25
VESTA.MX         25
BOLSAA.MX        25
MEGACPO.MX       25
GCARSOA1.MX      25
WALMEX.MX        25
PINFRA.MX        25
ALFAA.MX         25
PE&OLES.MX       25
CUERVO.MX        25
AC.MX            25
TLEVISACPO.MX    25
GMEXICOB.MX      25
FEMSAUBD.MX      25
GFNORTEO.MX      25
OMAB.MX          25
GAPB.MX          25
ELEKTRA.MX       25
CEMEXCPO.MX      25
ASURB.MX         25
GRUMAB.MX        25
BIMBOA.MX        25
KOFUBL.MX        25
ORBIA.MX         25
KIMBERA.MX       25
RA.MX            23
Name: Ticker, dtype: int64

In [1227]:
df_global_list = df_global['Ticker'].value_counts(sort=True,ascending=False).index[:34].tolist()

In [1228]:
df_global_list.remove('LIVEPOLC1.MX')
df_global_list.append('LIVEPOLC-1.MX')

# <font color= #6B6B6B> 4. Pasive investment  </font>

<hr style="border:0.02in solid gray"> </hr>

In [1229]:
p_portfolio = df_global.head(33)
p_portfolio['Precio'] = p_portfolio['Precio'].astype(str)
p_portfolio['Precio'] = p_portfolio['Precio'].map(lambda x: x.replace(',',''))
p_portfolio['Precio'] = p_portfolio['Precio'].astype(float)
p_portfolio['Peso (%)'] = p_portfolio['Peso (%)'].apply(lambda x: x/100)
p_portfolio


C:\Users\blas_\AppData\Local\Temp\ipykernel_16676\1688884738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_portfolio['Precio'] = p_portfolio['Precio'].astype(str)
C:\Users\blas_\AppData\Local\Temp\ipykernel_16676\1688884738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_portfolio['Precio'] = p_portfolio['Precio'].map(lambda x: x.replace(',',''))
C:\Users\blas_\AppData\Local\Temp\ipykernel_16676\1688884738.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,Ticker,Peso (%),Precio
0,AMXL.MX,0.1229,13.67
1,WALMEX.MX,0.1114,58.35
2,GMEXICOB.MX,0.1057,87.84
3,FEMSAUBD.MX,0.0979,139.70
4,GFNORTEO.MX,0.0939,101.57
5,CEMEXCPO.MX,0.0671,11.76
6,GAPB.MX,0.0308,206.60
7,ELEKTRA.MX,0.0305,1386.01
8,TLEVISACPO.MX,0.0296,30.99
9,ASURB.MX,0.0243,322.53


In [1230]:
p_portfolio = p_portfolio.sort_values('Ticker')
p_portfolio = p_portfolio.reset_index(drop=True)
p_portfolio

,Ticker,Peso (%),Precio
0,AC.MX,0.0155,93.07
1,ALFAA.MX,0.0136,12.90
2,AMXL.MX,0.1229,13.67
3,ASURB.MX,0.0243,322.53
4,BBAJIOO.MX,0.0067,24.76
5,BIMBOA.MX,0.0204,38.52
6,BOLSAA.MX,0.0082,46.00
7,CEMEXCPO.MX,0.0671,11.76
8,CUERVO.MX,0.0090,43.73
9,ELEKTRA.MX,0.0305,1386.01


In [1231]:
p_portfolio['Value'] = p_portfolio['Peso (%)'] * 1000000
january_2021 = p_portfolio.Value.sum()

In [1232]:
import yfinance as yf

tickers = df_global_list

data = yf.download(tickers = tickers, start= '2021-01-01', end='2023-02-01', interval = "1mo")

[*********************100%***********************]  34 of 34 completed

1 Failed download:
- MXN.MX: No timezone found, symbol may be delisted


In [1233]:
data = data['Adj Close']
data = data.transpose()
data

Date,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,...,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01
AC.MX,84.994209,86.546692,92.099129,99.048805,105.987480,108.406509,112.944542,121.298645,118.026382,119.139786,...,123.938515,131.361862,130.095276,138.647202,134.660873,142.378235,158.922424,157.685303,158.039993,166.100006
ALFAA.MX,12.846219,11.342515,11.750805,14.110926,13.945353,14.922925,15.122429,14.065057,14.274536,14.873049,...,13.670000,15.120000,14.280000,14.010000,12.820000,12.800000,13.160000,13.500000,12.430000,13.630000
ALSEA.MX,23.110001,25.440001,30.280001,32.950001,33.410000,35.470001,40.369999,39.259998,41.610001,43.220001,...,44.730000,42.150002,38.080002,39.660000,36.560001,36.150002,38.060001,38.799999,36.779999,45.020000
AMXL.MX,13.036207,12.769189,13.312761,13.493952,14.857652,14.304543,15.906651,19.092947,17.634661,17.721577,...,19.389942,20.317455,20.063608,18.940828,16.763609,16.590000,18.700001,18.750000,17.600000,19.660000
ASURB.MX,301.710938,366.163483,339.596680,320.494720,339.830536,344.143005,338.268341,337.164520,360.868805,396.972137,...,425.827881,411.084473,395.230011,384.000000,426.709991,396.000000,464.970001,478.500000,454.010010,510.230011
BBAJIOO.MX,19.837048,19.252193,22.088346,25.733683,27.712582,30.464661,30.362431,32.747807,33.224880,32.926712,...,42.928249,41.360718,37.080383,44.099068,38.653378,50.990002,55.779999,62.689999,61.380001,75.639999
BIMBOA.MX,36.873363,37.620018,41.066128,38.807011,41.975513,43.077446,45.030609,49.879150,56.965485,59.811794,...,61.735508,63.521816,64.881554,71.501709,65.725197,70.241196,76.236069,81.258263,82.300003,93.489998
BOLSAA.MX,41.553848,38.708313,37.262962,40.596302,40.198833,41.468040,38.370453,39.286507,37.511059,37.246632,...,38.181576,34.234035,35.689999,37.939999,36.389999,33.799999,35.919998,38.090000,37.730000,40.889999
CEMEXCPO.MX,11.760000,13.940000,14.420000,16.090000,16.809999,16.820000,16.240000,16.500000,14.890000,13.240000,...,8.980000,9.260000,7.830000,8.230000,7.470000,6.960000,7.720000,8.820000,7.920000,10.020000
CUERVO.MX,42.955780,42.386051,45.932133,47.454693,49.301407,51.511570,48.928135,50.283699,44.411697,46.631290,...,50.406582,43.926163,43.103722,45.719669,40.824677,35.529999,41.959999,43.480000,42.549999,45.639999


In [1234]:
data = data.reset_index(drop=True)
# Concatenate the two DataFrames
data

Date,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,...,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01
0,84.994209,86.546692,92.099129,99.048805,105.987480,108.406509,112.944542,121.298645,118.026382,119.139786,...,123.938515,131.361862,130.095276,138.647202,134.660873,142.378235,158.922424,157.685303,158.039993,166.100006
1,12.846219,11.342515,11.750805,14.110926,13.945353,14.922925,15.122429,14.065057,14.274536,14.873049,...,13.670000,15.120000,14.280000,14.010000,12.820000,12.800000,13.160000,13.500000,12.430000,13.630000
2,23.110001,25.440001,30.280001,32.950001,33.410000,35.470001,40.369999,39.259998,41.610001,43.220001,...,44.730000,42.150002,38.080002,39.660000,36.560001,36.150002,38.060001,38.799999,36.779999,45.020000
3,13.036207,12.769189,13.312761,13.493952,14.857652,14.304543,15.906651,19.092947,17.634661,17.721577,...,19.389942,20.317455,20.063608,18.940828,16.763609,16.590000,18.700001,18.750000,17.600000,19.660000
4,301.710938,366.163483,339.596680,320.494720,339.830536,344.143005,338.268341,337.164520,360.868805,396.972137,...,425.827881,411.084473,395.230011,384.000000,426.709991,396.000000,464.970001,478.500000,454.010010,510.230011
5,19.837048,19.252193,22.088346,25.733683,27.712582,30.464661,30.362431,32.747807,33.224880,32.926712,...,42.928249,41.360718,37.080383,44.099068,38.653378,50.990002,55.779999,62.689999,61.380001,75.639999
6,36.873363,37.620018,41.066128,38.807011,41.975513,43.077446,45.030609,49.879150,56.965485,59.811794,...,61.735508,63.521816,64.881554,71.501709,65.725197,70.241196,76.236069,81.258263,82.300003,93.489998
7,41.553848,38.708313,37.262962,40.596302,40.198833,41.468040,38.370453,39.286507,37.511059,37.246632,...,38.181576,34.234035,35.689999,37.939999,36.389999,33.799999,35.919998,38.090000,37.730000,40.889999
8,11.760000,13.940000,14.420000,16.090000,16.809999,16.820000,16.240000,16.500000,14.890000,13.240000,...,8.980000,9.260000,7.830000,8.230000,7.470000,6.960000,7.720000,8.820000,7.920000,10.020000
9,42.955780,42.386051,45.932133,47.454693,49.301407,51.511570,48.928135,50.283699,44.411697,46.631290,...,50.406582,43.926163,43.103722,45.719669,40.824677,35.529999,41.959999,43.480000,42.549999,45.639999


In [1235]:
data.insert(0, 'Peso (%)', p_portfolio['Peso (%)'])
data.insert(0, 'Ticker', p_portfolio['Ticker']) 

In [1236]:
data = data.dropna()
data

Date,Ticker,Peso (%),2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,...,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00
0,AC.MX,0.0155,84.994209,86.546692,92.099129,99.048805,105.987480,108.406509,112.944542,121.298645,...,123.938515,131.361862,130.095276,138.647202,134.660873,142.378235,158.922424,157.685303,158.039993,166.100006
1,ALFAA.MX,0.0136,12.846219,11.342515,11.750805,14.110926,13.945353,14.922925,15.122429,14.065057,...,13.670000,15.120000,14.280000,14.010000,12.820000,12.800000,13.160000,13.500000,12.430000,13.630000
2,AMXL.MX,0.1229,23.110001,25.440001,30.280001,32.950001,33.410000,35.470001,40.369999,39.259998,...,44.730000,42.150002,38.080002,39.660000,36.560001,36.150002,38.060001,38.799999,36.779999,45.020000
3,ASURB.MX,0.0243,13.036207,12.769189,13.312761,13.493952,14.857652,14.304543,15.906651,19.092947,...,19.389942,20.317455,20.063608,18.940828,16.763609,16.590000,18.700001,18.750000,17.600000,19.660000
4,BBAJIOO.MX,0.0067,301.710938,366.163483,339.596680,320.494720,339.830536,344.143005,338.268341,337.164520,...,425.827881,411.084473,395.230011,384.000000,426.709991,396.000000,464.970001,478.500000,454.010010,510.230011
5,BIMBOA.MX,0.0204,19.837048,19.252193,22.088346,25.733683,27.712582,30.464661,30.362431,32.747807,...,42.928249,41.360718,37.080383,44.099068,38.653378,50.990002,55.779999,62.689999,61.380001,75.639999
6,BOLSAA.MX,0.0082,36.873363,37.620018,41.066128,38.807011,41.975513,43.077446,45.030609,49.879150,...,61.735508,63.521816,64.881554,71.501709,65.725197,70.241196,76.236069,81.258263,82.300003,93.489998
7,CEMEXCPO.MX,0.0671,41.553848,38.708313,37.262962,40.596302,40.198833,41.468040,38.370453,39.286507,...,38.181576,34.234035,35.689999,37.939999,36.389999,33.799999,35.919998,38.090000,37.730000,40.889999
8,CUERVO.MX,0.0090,11.760000,13.940000,14.420000,16.090000,16.809999,16.820000,16.240000,16.500000,...,8.980000,9.260000,7.830000,8.230000,7.470000,6.960000,7.720000,8.820000,7.920000,10.020000
9,ELEKTRA.MX,0.0305,42.955780,42.386051,45.932133,47.454693,49.301407,51.511570,48.928135,50.283699,...,50.406582,43.926163,43.103722,45.719669,40.824677,35.529999,41.959999,43.480000,42.549999,45.639999


In [1237]:
primero = pd.DataFrame({'capital': [january_2021]})
primero

,capital
0,986700.0


In [1238]:
results=[]
for j in range (0,24):
    suma = 0
    for i in range (0, 31):
        X = 0
        X = (data.iloc[i,j+3]/data.iloc[i,j+2]-1)*data.iloc[i,1]
        if X>0:
            suma += X
    results.append(suma)

In [1239]:
df = pd.DataFrame (results, columns = ['rendimiento'])
df.loc[-1] = [0]  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 
df

,rendimiento
0,0.000000
1,0.032070
2,0.056865
3,0.046846
4,0.058732
5,0.024168
6,0.027467
7,0.019856
8,0.011742
9,0.021144


In [1240]:
import datetime
dates=[] 
# range of dates
date_range = pd.period_range(
    start='2021-01-01', periods=26, freq='M')
  
# timestamp range
timestamp_range = [x.to_timestamp() for x in date_range]
  
# iterating through timestamp range
for i in timestamp_range:
    dates.append(i)

df_pasiva = pd.DataFrame (dates, columns = ['timestamp'])

df_pasiva

,timestamp
0,2021-01-01
1,2021-02-01
2,2021-03-01
3,2021-04-01
4,2021-05-01
5,2021-06-01
6,2021-07-01
7,2021-08-01
8,2021-09-01
9,2021-10-01


In [1241]:
df_pasiva.insert(1, 'capital', primero['capital'])
df_pasiva.insert(2, 'rendimiento', df['rendimiento'])

In [1242]:
df_pasiva

,timestamp,capital,rendimiento
0,2021-01-01,986700.0,0.000000
1,2021-02-01,NaN,0.032070
2,2021-03-01,NaN,0.056865
3,2021-04-01,NaN,0.046846
4,2021-05-01,NaN,0.058732
5,2021-06-01,NaN,0.024168
6,2021-07-01,NaN,0.027467
7,2021-08-01,NaN,0.019856
8,2021-09-01,NaN,0.011742
9,2021-10-01,NaN,0.021144


In [1243]:
df_pasiva.iloc[1,1] = df_pasiva.iloc[0,1] #la primera es la fila y la segunda es la columna

for i in range (0,23):
    df_pasiva.iloc[i+2,1] = df_pasiva.iloc[i+1,1]*(1 + df_pasiva.iloc[i+1,2])

    
#df_pasiva


In [1244]:
df_pasiva

,timestamp,capital,rendimiento
0,2021-01-01,9.867000e+05,0.000000
1,2021-02-01,9.867000e+05,0.032070
2,2021-03-01,1.018344e+06,0.056865
3,2021-04-01,1.076252e+06,0.046846
4,2021-05-01,1.126670e+06,0.058732
5,2021-06-01,1.192842e+06,0.024168
6,2021-07-01,1.221670e+06,0.027467
7,2021-08-01,1.255227e+06,0.019856
8,2021-09-01,1.280151e+06,0.011742
9,2021-10-01,1.295182e+06,0.021144


In [1245]:
acumulado=[]
for i in range(0,23):
    x  = df_pasiva.iloc[i+1,1]/df_pasiva.iloc[0,1]-1
    acumulado.append(x)
acumulado = pd.DataFrame (acumulado, columns = ['rendimiento acumulado'])
acumulado
    

,rendimiento acumulado
0,0.000000
1,0.032070
2,0.090759
3,0.141857
4,0.208920
5,0.238138
6,0.272146
7,0.297406
8,0.312640
9,0.340394


In [1246]:
df_pasiva.insert(3, 'rendimiento acumulado', acumulado['rendimiento acumulado'])

In [1247]:
df_pasiva = df_pasiva.dropna()
df_pasiva


,timestamp,capital,rendimiento,rendimiento acumulado
0,2021-01-01,9.867000e+05,0.000000,0.000000
1,2021-02-01,9.867000e+05,0.032070,0.032070
2,2021-03-01,1.018344e+06,0.056865,0.090759
3,2021-04-01,1.076252e+06,0.046846,0.141857
4,2021-05-01,1.126670e+06,0.058732,0.208920
5,2021-06-01,1.192842e+06,0.024168,0.238138
6,2021-07-01,1.221670e+06,0.027467,0.272146
7,2021-08-01,1.255227e+06,0.019856,0.297406
8,2021-09-01,1.280151e+06,0.011742,0.312640
9,2021-10-01,1.295182e+06,0.021144,0.340394


In [1248]:
# DELETE THIS CELL IN YOUR FILE

# A particular process that is relevant for the work you are presenting.

<br>

# <font color= #6B6B6B> 5. Active investment  </font>

<hr style="border:0.02in solid gray"> </hr>

In [1249]:
def cartera_eficiente(n_pt,
n_pt
vec_rendimientos = []
vec_varianza = []
Total_weights = []
vec_sharpe = []
)

# Another particular process that is relevant for the work. Normally, you include between 2 and 5 at the most.

SyntaxError: invalid syntax (3459922577.py, line 3)

<br>

In [ ]:
for i in n_pt:
    weight = np.random(Tickers)
    weight = weights/sum(weights)

# <font color= #6B6B6B> <font size = 6> 5. Results </font>

<hr style="border:0.02in solid gray"> </hr>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# In here you present the general results, and also, a discussion of such results.

<br>

## <font color= #6B6B6B> 5.1 Results of first processes </font>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# Separate your results according to the main processes you are working on

<br>

## <font color= #6B6B6B> 5.2 Results of another proces </font>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# You can have several processes with which you are obtaining the results you need.

<br>

# <font color= #6B6B6B> <font size = 6> 6. References </font>

<hr style="border:0.02in solid gray"> </hr>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# Citations, and other references you are using.

# DO INCLUDE THE FOLLOWING CITATION FOR THIS TEMPLATE

[1] Munnoz, 2020. Python project template. https://github.com/iffranciscome/python-project. (2021).

<br>

# <font color= red> TESTS (DO NOT INCLUDE IN THE FINAL NOTEBOOK) </font>

<hr style="border:0.02in solid red"> </hr>

Next are a series of tests to verify that everything is working ok. From correctly installed and loaded packages, to import other scripts, print plots with plotly

In [ ]:
import pandas as pd
import data as dt

In [ ]:
# -- TEST 1 : 
# verify that the script is being read
print(dt.dict_test)

In [ ]:
# -- TEST 2 :
# verify that installed pandas module works correctly
df_dict_test = pd.DataFrame(dt.dict_test, index=[0,1])
print(df_dict_test)

In [ ]:
# -- TEST 3 :
# verify you can use plotly and visualize plots in jupyter notebook

from plotly.offline import iplot   # print the plot inside jupyter and offline
import plotly.graph_objects as go  # functions to create plots

# example data
df = pd.DataFrame({'column_a': [1, 2, 3, 4, 5], 'column_b': [1, 2, 3, 4, 5]})
# basic plotly plot
data = [go.Bar(x=df['column_a'], y=df['column_b'])]

# instruction to view it inside jupyter
iplot(data)

In [ ]:
# -- TEST 4 :
# verify you can use plotly and visualize plots in web browser locally

import plotly.io as pio            # to define input-output of plots
pio.renderers.default = "browser"  # to render the plot locally in your default web browser

# basic plotly plot
plot_data = go.Figure(go.Bar(x=df['column_a'], y=df['column_b']))

# instruction to view it in specified render (in this case browser)
plot_data.show()